In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
HOME = '/content/drive/MyDrive/20240422'

import os
%cd {HOME}

/content/drive/MyDrive/20240422


In [5]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from tensorflow import keras
from keras.applications.mobilenet_v2 import preprocess_input
from keras.models import load_model

In [6]:
!pip install ffmpeg-python

import ffmpeg
from base64 import b64encode
from IPython.display import HTML

def get_video_metadata(video_path):
    try:
        probe = ffmpeg.probe(video_path)
        video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
        return {
            'video_codec': video_stream['codec_name'] if video_stream else 'None',
        }
    except Exception as e:
        return {'error': str(e)}

def convert_video_to_h264(input_path, output_path):
    try:
        ffmpeg.input(input_path).output(output_path, vcodec='libx264').run(overwrite_output=True)
        return True
    except ffmpeg.Error as e:
        return False

def check_and_convert_video(video_path):
    metadata = get_video_metadata(video_path)

    if metadata.get('error'):
        return f"Error retrieving metadata: {metadata['error']}", 0

    if metadata['video_codec'] == 'mpeg4':
        output_path = video_path.replace('.mp4', '_converted.mp4')
        success = convert_video_to_h264(video_path, output_path)
        if success:
            return output_path, 1
        else:
            return "Failed to convert video.",0
    else:
        return video_path, 2

def play_video(video_path):

    play_video_file, result = check_and_convert_video(video_path)

    if result!=0:
        # ��������� ��������� ������ base64��� ���������
        try:
            with open(play_video_file, "rb") as video_file:
                video_encoded = b64encode(video_file.read()).decode('ascii')

            # HTML ������ ������
            video_html = f'''
            <video width=800 controls>
            <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
            </video>
            '''
            display(HTML(video_html))
        except Exception as e:
            print(f"Error: {e}")
    else :
        print("unknown file!!!")

In [ ]:
facenet = cv2.dnn.readNet('models/deploy.prototxt', 'models/res10_300x300_ssd_iter_140000.caffemodel')
model = load_model('models/mask_detector.model')

cap = cv2.VideoCapture('videos/mask.mp4')

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

out = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while cap.isOpened():
  ret, img = cap.read()

  if not ret:
    break

  h, w, c = img.shape

  blob = cv2.dnn.blobFromImage(img, size = (300, 300), mean = (104., 177., 123.))

  facenet.setInput(blob)
  dets = facenet.forward()

  for i in range(dets.shape[2]):
    confidence = dets[0, 0, i, 2]

    if confidence < 0.5:
      continue

    x1 = int(dets[0, 0, i, 3] * w)
    y1 = int(dets[0, 0, i, 4] * h)
    x2 = int(dets[0, 0, i, 5] * w)
    y2 = int(dets[0, 0, i, 6] * h)

    face = img[y1:y2, x1:x2]

    face_input = cv2.resize(face, dsize=(224,224))
    face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
    face_input = preprocess_input(face_input)
    face_input = np.expand_dims(face_input, axis=0)

    mask, nomask = model.predict(face_input).squeeze()

    if mask > nomask :
      color = (0, 255, 0)
    else:
      color = (0, 0, 255)
      cv2.putText(img, text = '%', fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 1, color = (0, 255, 0), thickness = 2)

    cv2.rectangle(img, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color)

  out.write(img)

out.release()

play_video('output.mp4')

Output hidden; open in https://colab.research.google.com to view.

In [8]:
facenet = cv2.dnn.readNet('models/deploy.prototxt', 'models/res10_300x300_ssd_iter_140000.caffemodel')
model = load_model('models/mask_detector.model')

cap = cv2.VideoCapture('videos/mask3.mp4')

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

out = cv2.VideoWriter('output1.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while cap.isOpened():
  ret, img = cap.read()

  if not ret:
    break

  h, w, c = img.shape

  blob = cv2.dnn.blobFromImage(img, size = (300, 300), mean = (104., 177., 123.))

  facenet.setInput(blob)
  dets = facenet.forward()

  for i in range(dets.shape[2]):
    confidence = dets[0, 0, i, 2]

    if confidence < 0.5:
      continue

    x1 = int(dets[0, 0, i, 3] * w)
    y1 = int(dets[0, 0, i, 4] * h)
    x2 = int(dets[0, 0, i, 5] * w)
    y2 = int(dets[0, 0, i, 6] * h)

    face = img[y1:y2, x1:x2]

    face_input = cv2.resize(face, dsize=(224,224))
    face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
    face_input = preprocess_input(face_input)
    face_input = np.expand_dims(face_input, axis=0)

    mask, nomask = model.predict(face_input).squeeze()

    if mask > nomask :
      color = (0, 255, 0)
      label = 'Mask %d%%' % (mask * 100)
    else:
      color = (0, 0, 255)
      label = 'nomask %d%%' % (nomask * 100)

    cv2.putText(img, text = label, org = (x1, y1), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 1, color = (color), thickness = 2)

    cv2.rectangle(img, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color)

  out.write(img)

out.release()

play_video('output1.mp4')

Output hidden; open in https://colab.research.google.com to view.